In [6]:
!jupyter nbconvert --to script ConvRNN.ipynb
import ConvRNN

[NbConvertApp] Converting notebook ConvRNN.ipynb to script
[NbConvertApp] Writing 8740 bytes to ConvRNN.py


SyntaxError: invalid syntax (ConvRNN.py, line 195)